In [1]:
import os
os.chdir('applications/vimc_example')

import sys
sys.path.append('applications/vimc_example')

import pandas as pd
import numpy as np
import atomica as at

In [2]:
import vimc_functions as vimc
import hbv_functions as hbv

In [8]:
fw_loc = 'C:/Users/Phil/Documents/GitHub/hbv-globalinvcase/frameworks/hbv_v15_csdev_b.xlsx'
db = f"C:/Users/Phil/Documents/GitHub/hbv-globalinvcase/applications/vimc_WPR_test/WPR_db.xlsx"
cl = f"C:/Users/Phil/Documents/GitHub/hbv-globalinvcase/applications/vimc_WPR_test/WPR_test_calib.xlsx"

In [9]:
P, res = vimc.load_cen_project(fw= fw_loc, db=db, cl=cl)

Parameter "m_dc" is in rate units and a maximum value of "1" has been entered. Rates in the framework should generally not be limited to "1"
Parameter "m_hcc" is in rate units and a maximum value of "1" has been entered. Rates in the framework should generally not be limited to "1"


Elapsed time for running "default": 6.09s


In [11]:
main = 'C:/Users/Phil/Documents/GitHub/hbv-globalinvcase/'

In [13]:
loc = 'applications/vimc_example/Data/Templates/' # The location for the output templates may be different
loc2 = 'applications/vimc_example/Data/Demographics/' # The location for the input data may be different (depending on where you store it)
df1 = pd.read_csv(main+loc2+'202212rfp-1_dds-202208_int_pop_both.csv')

In [15]:
output_dict = {'cohort_size': 'alive', 'cases': 'tot_inc', 'dalys':'dalys'}
cen_df = pd.read_csv(main+loc+'/central-burden-template.202212rfp-1.RFP_standard template_HepB.csv')

In [20]:
df1.age_from.unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100], dtype=int64)

In [21]:
df1 = df1[(df1.year >= 1990) & (df1.year <=2100)]
age_groups = ['0-4', '5-9','10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44',\
              '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89',\
              '90+']

## Assort data by age group
for idx,row in df1.iterrows():
    if row.age_from < 5:
        df1.loc[idx,'age_group'] = '0-4'
    elif row.age_from < 10:
        df1.loc[idx,'age_group'] = '5-9'
    elif row.age_from < 15:
        df1.loc[idx,'age_group'] = '10-14'
    elif row.age_from < 20:
        df1.loc[idx,'age_group'] = '15-19'
    elif row.age_from < 25:
        df1.loc[idx,'age_group'] = '20-24'
    elif row.age_from < 30:
        df1.loc[idx,'age_group'] = '25-29'
    elif row.age_from < 35:
        df1.loc[idx,'age_group'] = '30-34'
    elif row.age_from < 40:
        df1.loc[idx,'age_group'] = '35-39'
    elif row.age_from < 45:
        df1.loc[idx,'age_group'] = '40-44'
    elif row.age_from < 50:
        df1.loc[idx,'age_group'] = '45-49'
    elif row.age_from < 55:
        df1.loc[idx,'age_group'] = '50-54'
    elif row.age_from < 60:
        df1.loc[idx,'age_group'] = '55-59'
    elif row.age_from < 65:
        df1.loc[idx,'age_group'] = '60-64'
    elif row.age_from < 70:
        df1.loc[idx,'age_group'] = '65-69'
    elif row.age_from < 75:
        df1.loc[idx,'age_group'] = '70-74'
    elif row.age_from < 80:
        df1.loc[idx,'age_group'] = '75-79'
    elif row.age_from < 85:
        df1.loc[idx,'age_group'] = '80-84'
    elif row.age_from < 90:
        df1.loc[idx,'age_group'] = '85-89'
    else:
        df1.loc[idx,'age_group'] = '90+'

In [27]:
for opt in output_dict:
    print(opt)
    age_r = 0
    while age_r < 85:
        tot_val = res.get_variable(output_dict[opt])[int(age_r/5)].vals + res.get_variable(output_dict[opt])[int(age_r/5+1)].vals
        for age in range(age_r,age_r+5):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[(f'{age_r}-{age_r+4}',year),'value'] # Ratio of age relative to age group
                res_val = tot_val[(year-1990)*4] # The '4' should be timestep (if future runs are different)

                idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0] #Index of age group year sim entry
                cen_df.loc[idx, opt] = ratio * res_val
        age_r += 5
                
    '''
    tot_val = res.get_variable(output_dict[opt])[2].vals + res.get_variable(output_dict[opt])[3].vals
    for age in range(5,15):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('5-14',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, opt] = ratio * res_val

    tot_val = res.get_variable(output_dict[opt])[4].vals + res.get_variable(output_dict[opt])[5].vals
    for age in range(15,50):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('15-49',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, opt] = ratio * res_val

    tot_val = res.get_variable(output_dict[opt])[6].vals + res.get_variable(output_dict[opt])[7].vals
    for age in range(50,70):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('50-69',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, opt] = ratio * res_val
    '''
    tot_val = res.get_variable(output_dict[opt])[-2].vals + res.get_variable(output_dict[opt])[-1].vals
    for age in range(90,101):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_from == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('90+',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, opt] = ratio * res_val
            


cohort_size


KeyError: ('90-94', 2000)

In [28]:
age_r

90

In [ ]:
tot_val = 0
for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
    tot_val += res.get_variable(var)[0].vals + res.get_variable(var)[1].vals # Total deaths in age group

for age in range(0,5):
    for year in range(2000,2101):
        ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('0-4',year),'value']
        res_val = tot_val[(year-1990)*4]

        idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
        cen_df.loc[idx, 'deaths'] = ratio * res_val
            
tot_val = 0
for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
    tot_val += res.get_variable(var)[-2].vals + res.get_variable(var)[-1].vals
for age in range(90,101):
    for year in range(2000,2101):
        ratio = float(df1[(df1.age_from == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('90+',year),'value']
        res_val = tot_val[(year-1990)*4]

        idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
        cen_df.loc[idx, 'deaths'] = ratio * res_val

### Central Results Function

In [ ]:
def central_results(fw = 'hbv_v14_gamma_2.xlsx', db = "AFR_db_v1_2_1.xlsx", cl = 'AFR_calib_v1_2.xlsx'):
    '''
    Created by: Phillip Luong (https://github.com/phillipluong)
    Last Updated: 31/01/23
    Generates a DataFrame listing all central output results.

    Inputs:
    - fw:       Excel sheet of the Atomica Framework of the model (str)
    - db:       Excel sheet of the Atomica Databook of the model (str)
    - cl:       Excel sheet of the Model Calibration of the model - specific to a region estimate (str)

    Outputs:
    - cen_df:   DataFrame listing the central estimates of 1-year age groups between 2000-2100 (DataFrame)
    '''
    P, res = load_cen_project(fw= fw, db=db, cl=cl)
    ## Locations for the important dataframes
    loc = 'applications/vimc/Data/Templates/' # The location for the output templates may be different
    loc2 = 'applications/vimc/Data/Demographics/' # The location for the input data may be different (depending on where you store it)

    ## Load and process the input data
    df1 = pd.read_csv(loc2+'202212rfp-1_dds-202208_int_pop_both.csv')

    df1 = df1[(df1.year >= 1990) & (df1.year <=2100)]
    age_groups = ['0-4', '5-14', '15-49', '50-69', '70+']

    ## Assort data by age group
    for idx,row in df1.iterrows():
        if row.age_from < 5:
            df1.loc[idx,'age_group'] = '0-4'
        elif row.age_from < 15:
            df1.loc[idx,'age_group'] = '5-14'
        elif row.age_from < 50:
            df1.loc[idx,'age_group'] = '15-49'
        elif row.age_from < 70:
            df1.loc[idx,'age_group'] = '50-69'
        else:
            df1.loc[idx,'age_group'] = '70+'


    ## Start generating output results
    output_dict = {'cohort_size': 'alive', 'cases': 'tot_inc', 'dalys':'dalys'}
    cen_df = pd.read_csv(loc+'/central-burden-template.202212rfp-1.RFP_standard template_HepB.csv')

    for opt in output_dict:
        tot_val = res.get_variable(output_dict[opt])[0].vals + res.get_variable(output_dict[opt])[1].vals
        for age in range(0,5):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('0-4',year),'value'] # Ratio of age relative to age group
                res_val = tot_val[(year-1990)*4] # The '4' should be timestep (if future runs are different)

                idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0] #Index of age group year sim entry
                cen_df.loc[idx, opt] = ratio * res_val

        tot_val = res.get_variable(output_dict[opt])[2].vals + res.get_variable(output_dict[opt])[3].vals
        for age in range(5,15):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('5-14',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
                cen_df.loc[idx, opt] = ratio * res_val

        tot_val = res.get_variable(output_dict[opt])[4].vals + res.get_variable(output_dict[opt])[5].vals
        for age in range(15,50):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('15-49',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
                cen_df.loc[idx, opt] = ratio * res_val

        tot_val = res.get_variable(output_dict[opt])[6].vals + res.get_variable(output_dict[opt])[7].vals
        for age in range(50,70):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('50-69',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
                cen_df.loc[idx, opt] = ratio * res_val

        tot_val = res.get_variable(output_dict[opt])[8].vals + res.get_variable(output_dict[opt])[9].vals
        for age in range(70,101):
            for year in range(2000,2101):
                ratio = float(df1[(df1.age_from == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('70+',year),'value']
                res_val = tot_val[(year-1990)*4]

                idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
                cen_df.loc[idx, opt] = ratio * res_val

    ## Deaths are calculated separately
    tot_val = 0
    for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
        tot_val += res.get_variable(var)[0].vals + res.get_variable(var)[1].vals # Total deaths in age group

    for age in range(0,5):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('0-4',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, 'deaths'] = ratio * res_val

    tot_val = 0
    for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
        tot_val += res.get_variable(var)[2].vals + res.get_variable(var)[3].vals
    for age in range(5,15):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('5-14',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, 'deaths'] = ratio * res_val

    tot_val = 0
    for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
        tot_val += res.get_variable(var)[4].vals + res.get_variable(var)[5].vals
    for age in range(15,50):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('15-49',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, 'deaths'] = ratio * res_val

    tot_val = 0
    for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
        tot_val += res.get_variable(var)[6].vals + res.get_variable(var)[7].vals
    for age in range(50,70):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_to == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('50-69',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, 'deaths'] = ratio * res_val

    tot_val = 0
    for var in ['cl_acu', 'cl_cir', 'cl_hcc']:
        tot_val += res.get_variable(var)[8].vals + res.get_variable(var)[9].vals
    for age in range(70,101):
        for year in range(2000,2101):
            ratio = float(df1[(df1.age_from == age) & (df1.year == year)].value)/df1.groupby(by=['age_group','year']).sum().loc[('70+',year),'value']
            res_val = tot_val[(year-1990)*4]

            idx = cen_df[(cen_df.year == year) & (cen_df.age == age)].index[0]
            cen_df.loc[idx, 'deaths'] = ratio * res_val

    return cen_df